# MODELOS

#### Importar datos y explorar 

In [1]:
import numpy as np
import pandas as pd

In [2]:
#leer archivo limpio 

df = pd.read_csv('airbnb_rio_clean.csv')
#explorar df
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'airbnb_rio_clean.csv'

In [12]:
print(df.shape)
print( df.dtypes)
print(df.keys())

(36901, 34)
host_response_rate               int64
host_acceptance_rate             int64
host_is_superhost              float64
host_total_listings_count      float64
host_identity_verified         float64
latitude                       float64
longitude                      float64
room_type                      float64
accommodates                     int64
bedrooms                       float64
beds                           float64
price                          float64
minimum_nights                   int64
maximum_nights                   int64
maximum_minimum_nights         float64
maximum_nights_avg_ntm         float64
availability_365                 int64
availability_eoy                 int64
estimated_occupancy_l365d        int64
review_scores_rating           float64
review_scores_communication    float64
review_scores_location         float64
instant_bookable                 int64
bathroomsf                     float64
has_pool                         int64
has_wifi     

#### Definir variable dependiente del modelo

In [11]:
#Modelo de regresion--> variable y = precio . LISTA de nombre
y_reg= ["price"]

#Modelo de clasificacion --> variable y = rating . LISTA de nombre
y_clas= ["review_scores_rating"]

## MODELO DE REGRESIÓN

#### Exploracion de variables X importante

In [ ]:
# Histogramas de cada variable y dispersion entre variables 
import seaborn as sns
sns.pairplot(df,)


In [ ]:

#Ver correlacion entre datos 
corr = df.corr()[y_reg].sort_values(ascending=False)
sb.heatmap(corr, cmap="Blues", annot=True)

# Ver dispersion y tendencia entre variables x y y
#Definir variables x totales. LISTA
x_completo_reg=['host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'availability_365', 'availability_eoy',
       'estimated_occupancy_l365d',
       'review_scores_communication', 'review_scores_location',
       'instant_bookable', 'bathroomsf', 'has_pool', 'has_wifi',
       'has_free_parking', 'has_ac_heating', 'has_kitchen', 'has_jacuzzi',
       'has_washer_dryer', 'has_self_checkin', 'has_tv_cable', 'has_bbq']

sns.pairplot(df,x_vars= x_completo_reg, y_vars=y_reg, height=7, aspect=0.7, kind='reg')

KeyboardInterrupt: 

In [ ]:
#ANOVA para ver relevancia estadistica de variables 

import statsmodels.api as sm
from statsmodels.formula.api import ols

modelo = ols("price ~ C(neighbourhood_cleansed)", data=df).fit()
sm.stats.anova_lm(modelo, typ=2) 


#### Seleccionar variables x iniciales

In [ ]:

df.key()
#nombrar lista de variables x que se consideran relevantes
x_relevante_reg=[]

x
#crear nuevo dataframe con variables relevantes
datos=df.[[y,x_relevante_reg]].copy()

#Dividir variables x numericas
x_num=[]
#Dividir variables x categoricas
x_cat=[]

#### Dividir datos 

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Listas de tus variables
columnas_numericas = ["accommodates", "bedrooms", "bathrooms", "minimum_nights"]
columnas_categoricas = ["neighbourhood_cleansed", "room_type"]

# Definir el preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), columnas_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), columnas_categoricas),
    ]
)

#### Codificar variables dummies


In [ ]:
data = pd.get_dummies(df, columns=['Origin'], prefix='', prefix_sep='', dtype=int)

### Modelos de regresion- precios

### Modelos de clasificacion- alquilar o no

In [ ]:
print ()